In [1]:
! pip install --upgrade imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=8ede2f1d55755f52f1b48d11f5f5cca2c2576d31884945a6921d20f832e54646
  Stored in directory: /root/.cache/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
from imutils.video import FileVideoStream
from imutils.video import FPS
from imutils import face_utils
import numpy as np
import imutils
import time
import cv2
import pandas as pd
import os
from queue import Queue
import matplotlib.pyplot as plt
import dlib

In [3]:
SIZE = (128, 128)
WHITE = (255,255,255)
np.random.seed(42)

def create_optical_flow_img(landmarks1, landmarks2, show=False) : 
    optical_flow = np.zeros((SIZE[0],SIZE[1],3),dtype = np.uint8)
    for i in range(len(landmarks1)) :
        lm1 = tuple(landmarks1[i])
        lm2 = tuple(landmarks2[i])
        optical_flow = cv2.line(optical_flow, lm1, lm2, WHITE, 1)
    if show :
        plt.imshow(optical_flow)
        plt.show()
    return optical_flow

# l1 = np.random.randint(low=0, high=SIZE[0], size=(17,2))
# l2 = np.random.randint(low=0, high=SIZE[0], size=(17,2))
# sanity = create_optical_flow_img(l1, l2, True)

In [4]:
!mkdir Images


img_dir = './Images'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(
    '../input/facealignmentmodels/shape_predictor_68_face_landmarks.dat'
)

def get_data_frame(path, length=60, sample=2, distance=180, save=False) :
    fvs = FileVideoStream(path).start()
    fps = FPS().start()
    count = 0
    label = int(path.split('/')[-1][:-4])
    data_points = []
    optical_flows = []
    read = True
    frame_read = 0
    frame_count = 0
    last_read = 0
    data_count = 0
    prev_landmarks = None
    while fvs.more() :
        frame = fvs.read()
        if frame is None :
            break
        if read :
            if (frame_count - last_read) % sample == 0 or frame_read == 0 :
                # TO-DO: 
                # Face extraction
                try :
                    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                except :
                    print(frame_count)
                    print(frame)
                    plt.imshow(frame)
                    plt.show()
                faces = detector(gray, 0)
                if (len(faces) == 0) :
                    frame_read = 0
                    frame_count += 1
                    continue
                idx = np.argmax([f.area() for f in faces])
                face = faces[idx]
                landmarks = predictor(gray, face)
                landmarks = face_utils.shape_to_np(landmarks)[18:,:].reshape(-1)
                tl = face.tl_corner()
                br = face.br_corner()
                landmarks = np.hstack([[
                    data_count,
                    frame_count,
                    path,
                    label,
                    tl.x, 
                    tl.y, 
                    br.x, 
                    br.y
                ],landmarks])
                
                filename = path + '_' + str(frame_count) + '.jpg'
                cv2.imwrite(os.path.join(img_dir, filename), frame)
                
                if frame_read == 0 :
                    data_point = landmarks
                else :
                    data_point = np.vstack([data_point, landmarks])

                last_read = frame_count
                frame_read += 1
                if frame_read == length :
                    read = False
                    frame_read = 0
                    data_count += 1
                    data_points.append(data_point)
        else :
            if frame_count - last_read == distance - 1 :
                read = True
        frame_count += 1
        fps.update()
    fps.stop()
    fvs.stop()
    return np.vstack(data_points)

In [5]:
landmark_ids = []
for i in range(18,68) :
    landmark_ids.append('l_' + str(i) + '_x')
    landmark_ids.append('l_' + str(i) + '_y')
columns = [
    'id',
    'frame',
    'file',
    'label',
    'x_min',
    'y_min',
    'x_max',
    'y_max'
]
columns += landmark_ids
len(columns)

108

In [6]:
from tqdm import tqdm

FOLD1_1 = '../input/drowsiness-dataset-fold1-1/Fold1_part1'
ids = os.listdir('../input/drowsiness-dataset-fold1-1/Fold1_part1')
data_list = []
for idx in ids :
    folder = os.path.join(FOLD1_1, idx)
    files = os.listdir(folder)
    for file in tqdm(files) :
        print(os.path.join(folder,file))
        datas = get_data_frame(os.path.join(folder, file))
        data_list.append(datas)

data_list = np.vstack(data_list)
df = pd.DataFrame(data_list)
df.columns = columns

df.to_csv('Fold_1_1.csv')

  0%|          | 0/3 [00:00<?, ?it/s]

../input/drowsiness-dataset-fold1-1/Fold1_part1/05/10.MOV


 33%|███▎      | 1/3 [10:10<20:21, 610.64s/it]

../input/drowsiness-dataset-fold1-1/Fold1_part1/05/0.MOV


 67%|██████▋   | 2/3 [20:31<10:16, 616.72s/it]

...

 33%|███▎      | 1/3 [20:10<40:20, 1210.49s/it]

../input/drowsiness-dataset-fold1-1/Fold1_part1/01/0.mov


 67%|██████▋   | 2/3 [41:52<21:04, 1264.34s/it]

../input/drowsiness-dataset-fold1-1/Fold1_part1/01/5.mov


100%|██████████| 3/3 [1:02:16<00:00, 1245.37s/it]
